In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 불러오기

In [2]:
df=pd.read_csv('/content/drive/MyDrive/ml-100k/movie_lens.csv',index_col=0)
df_c = df.astype(dtype='int32',errors='ignore')
df_c.iloc[:,5:7]=df_c.iloc[:,5:7].astype('category')
df_c.iloc[:,-1:]=df_c.iloc[:,-1:].astype('category')

In [3]:
unknown = df_c['movie_title'] !='unknown'
df_c = df_c[unknown]

In [ ]:
df_c.info()

- 데이터 분리

In [5]:
from sklearn.model_selection import train_test_split

train, test=train_test_split(df_c, test_size=0.2, shuffle=True,random_state=3)

# CF - 아이템 기반 협업 필터링(코사인유사도)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import operator

from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity

from scipy.stats import pearsonr
from scipy.sparse import csr_matrix

In [7]:
train_data=train.pivot_table('rating',index='movie_title',columns='user_id').fillna(0)


sparse(0) data가 많기 때문에 scipy sparse matrix를 사용해 memory 용량 효율적으로 사용<BR>
- Compressed sparse row (CSR): 가로의 순서대로 재정렬하는 방법으로 행에 관여하여 정리 압축


In [8]:
train_csr=csr_matrix(train_data.values) 

### 코사인 유사도

In [9]:
train_cos = cosine_similarity(train_csr,train_csr)
item_cf = pd.DataFrame(data=train_cos, index=train_data.index, columns=train_data.index)

In [10]:
item_cf

movie_title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),Á köldum klaka (Cold Fever) (1994)
movie_title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),1.000000,0.000000,0.026928,0.082944,0.224786,0.122305,0.000000,0.034144,0.000000,0.037892,...,0.076208,0.000000,0.000000,0.000000,0.031298,0.127585,0.167428,0.030980,0.000000,0.000000
1-900 (1994),0.000000,1.000000,0.017695,0.000000,0.010187,0.000000,0.000000,0.035899,0.000000,0.000000,...,0.000000,0.000000,0.019462,0.000000,0.041134,0.000000,0.000000,0.008143,0.092319,0.000000
101 Dalmatians (1996),0.026928,0.017695,1.000000,0.147744,0.011897,0.085751,0.150920,0.227680,0.033884,0.075124,...,0.185204,0.000000,0.026518,0.033884,0.252874,0.149048,0.050989,0.066571,0.173706,0.000000
12 Angry Men (1957),0.082944,0.000000,0.147744,1.000000,0.078722,0.134049,0.241612,0.359520,0.092773,0.304385,...,0.171252,0.069580,0.020745,0.000000,0.304621,0.236066,0.145191,0.042312,0.045100,0.092773
187 (1997),0.224786,0.010187,0.011897,0.078722,1.000000,0.106072,0.012925,0.066376,0.058521,0.012556,...,0.056117,0.000000,0.170113,0.000000,0.057617,0.065764,0.088063,0.057487,0.031035,0.117041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Guns (1988),0.127585,0.000000,0.149048,0.236066,0.065764,0.211928,0.212745,0.247752,0.048162,0.093003,...,0.304814,0.000000,0.093335,0.000000,0.328770,1.000000,0.527621,0.079603,0.000000,0.000000
Young Guns II (1990),0.167428,0.000000,0.050989,0.145191,0.088063,0.187328,0.070903,0.133442,0.000000,0.099015,...,0.152960,0.180579,0.000000,0.000000,0.250093,0.527621,1.000000,0.008447,0.000000,0.000000
"Young Poisoner's Handbook, The (1995)",0.030980,0.008143,0.066571,0.042312,0.057487,0.131182,0.110203,0.118172,0.023389,0.045165,...,0.031400,0.000000,0.015690,0.000000,0.112377,0.079603,0.008447,1.000000,0.024808,0.000000


### 아이템 추천
- 아이템(영화 제목)과 유사도가 높은 25개 출력
- 아이템이 없다면 경고 메세지 나오게 함

In [11]:
def train_get_item_cf(title):
  try:
    return item_cf[title].sort_values(ascending=False)[1:26].reset_index()

  except:
    print('Sorry, the movie is not in the database!')

In [12]:
train_get_item_cf('Godfather, The (1972)')

,movie_title,"Godfather, The (1972)"
0,Star Wars (1977),0.576435
1,"Godfather: Part II, The (1974)",0.551052
2,Return of the Jedi (1983),0.499487
3,Fargo (1996),0.497635
4,One Flew Over the Cuckoo's Nest (1975),0.475017
5,"Silence of the Lambs, The (1991)",0.455092
6,Raiders of the Lost Ark (1981),0.454788
7,Pulp Fiction (1994),0.449306
8,"Usual Suspects, The (1995)",0.439788
9,Forrest Gump (1994),0.435817


## 성능평가

- 사용자가 제일 고평가한 최근 영화와 유사도가 비슷한 영화 뽑기

In [15]:
def movie_title(id,df):
    user_love =df[(df['user_id']==id) & (df['rating']==max(df['rating']))][['user_id','movie_title','timestamp']]
    user_love = user_love.sort_values('timestamp',ascending=False)
    return user_love.iloc[0]['movie_title']

### precision, recall 구하기
  - why 4점 이상인가
    - 별점은 주관적임
    - 대부분은 1점(매우 불만족), 2점(불만족), 3점(보통), 4점(만족), 5점(매우 만족) 이 기준으로 생각하기 때문에 만족(4점)이상을 선택함
  

In [16]:
def train_score(user_id,title):
  count=0
  # 유저가 4점이상 평가한 영화들 중에 최근에 별점이 높았던 영화의 precision,recall
  user_love= train_data[user_id][train_data[user_id]>=4].reset_index()['movie_title']
  user_recommend = train_get_item_cf(title)['movie_title']
  for i in user_love:
    for j in user_recommend: # 영화 유사도가 높은 25개
      if i ==j:
        count+=1
  
  precision = count / len(user_recommend)
  if len(user_love)==0:
    pass
  else: 
    recall = count/len(user_love)

  return precision,recall

In [17]:
train_score(12,movie_title(12,train))

(0.56, 0.358974358974359)

#### 전체(mean) precision@k

In [18]:
# precision@25
def precision(n):
  precision=0
  total = 0
  for i in range(1,n+1):
    #데이터 분리로 없는 user_id가 있어 try, except으로 나눠줌
    try:
      total+=1
      precision += train_score(i,movie_title(i,train))[0]
    except:
      pass
  mean_precision = precision/total
  return mean_precision

In [19]:
precision(30)

0.276

#### 전체(mean) recall@k

In [20]:
# recall@25
def recall(n):
  recall=0
  total = 0
  for i in range(1,n+1):
    try:
      total+=1
      recall += train_score(i,movie_title(i,train))[1]
    except:
      pass
  mean_recall = recall/total
  return mean_recall

In [21]:
recall(30)

0.15449145516044238

#### test

In [22]:
test_data= test.pivot_table('rating',index='movie_title',columns='user_id').fillna(0)
test_csr=csr_matrix(test_data.values) 
test_cos = cosine_similarity(test_csr,test_csr)
test_item_cf = pd.DataFrame(data=test_cos, index=test_data.index, columns=test_data.index)

In [23]:
def test_get_item_cf(title):
  try:
    return test_item_cf[title].sort_values(ascending=False)[1:26].reset_index()

  except:
    print('Sorry, the movie is not in the database!')

In [24]:
def test_score(user_id,title):
  count=0
  # 유저가 4점이상 평가한 영화들 중에 최근에 별점이 높았던 영화의 precision,recall
  user_love= test_data[user_id][test_data[user_id]>=4].reset_index()['movie_title']
  for i in user_love:
    for j in test_get_item_cf(title)['movie_title']: # 영화 유사도가 높은 25개
      if i ==j:
        count+=1
  
  precision = count / len(test_get_item_cf(title)['movie_title'])
  if len(user_love)==0:
    pass
  else: 
    recall = count/len(user_love)

  return precision,recall

In [25]:
test_score(12,movie_title(12,test))

(0.08, 0.25)

In [26]:
# precision@10
def precision(n):
  precision=0
  total = 0
  for i in range(1,n+1):
    #데이터 분리로 없는 user_id가 있어 try, except으로 나눠줌
    try:
      total+=1
      precision += test_score(i,movie_title(i,test))[0]
    except:
      pass
  mean_precision = precision/total
  return mean_precision

# recall@10
def recall(n):
  recall=0
  total = 0
  for i in range(1,n+1):
    try:
      total+=1
      recall += test_score(i,movie_title(i,test))[1]
    except:
      pass
  mean_recall = recall/total
  return mean_recall

In [27]:
print(precision(30),recall(30))

0.08266666666666668 0.12366641409211067


---------------------------